In [ ]:
pip install opencv-python cvzone autopy numpy


In [ ]:
import cv2
import numpy as np
import autopy
from cvzone.HandTrackingModule import HandDetector
import time

In [ ]:
# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

In [ ]:
# Initialize hand detector
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Get screen size
screen_w, screen_h = autopy.screen.size()

# Click state to avoid continuous clicking
click_down = False

# Smoothening parameters
plocX, plocY = 0, 0
clocX, clocY = 0, 0
smoothening = 7

In [ ]:
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    if hands:
        hand = hands[0]
        lmList = hand["lmList"]
        x1, y1 = lmList[8][0], lmList[8][1]   
        x2, y2 = lmList[6][0], lmList[6][1]   
        cv2.circle(img, (x1, y1), 10, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 10, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        screen_x = np.interp(x1, [100, 540], [0, screen_w])
        screen_y = np.interp(y1, [100, 380], [0, screen_h])
        clocX = plocX + (screen_x - plocX) / smoothening
        clocY = plocY + (screen_y - plocY) / smoothening
        clocX = max(0, min(screen_w - 1, clocX))
        clocY = max(0, min(screen_h - 1, clocY))
        autopy.mouse.move(clocX, clocY)
        plocX, plocY = clocX, clocY
        if y1 > y2 + 20:  
            if not click_down:
                autopy.mouse.click()
                click_down = True
                cv2.circle(img, (x1, y1), 15, (0, 255, 0), cv2.FILLED)
                cv2.putText(img, "Click!", (x1+20, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
        else:
            click_down = False
    cv2.putText(img, "Virtual Mouse | Bend index finger to click | 'q' to quit", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    cv2.imshow("Virtual Mouse", img)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
